<a href="https://colab.research.google.com/github/sainijagjit/CS6910-A2/blob/main/Part%20A/Assignment_2_iNaturalist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics
import pytorch_lightning as L
import wandb
from torch.utils.data import DataLoader, random_split, SubsetRandomSampler, Subset
from sklearn.model_selection import train_test_split
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import torchvision

from pytorch_lightning.callbacks import EarlyStopping, Callback, ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger

import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torchmetrics import Accuracy


In [ ]:
wandb.login(key="Your WandB api key")
wandb_project="DLASSIGN_2"
wandb_entity="ch22m009"
sweep_config = {
  "name": "DLA2",
  "method": "random",
  'metric': {
      'name': 'validation accuracy',
      'goal': 'maximize'
    },
  "parameters": {
        "LearningRate": {
            "values": [1e-3,1e-4,1e-5]

        },
        "Epochs": {
            "values": [10,15]

        },
        "NumDenseNeurons": {
            "values": [512]

        },
        "DropOut":{
            "values": [0,0.2,0.3,0.5]
        },
        "Filters": {
            "values": [[32,64,128,256,512],[64,64,128,128,256],[32,64,64,128,128],[128,128,64,64,32],[128,128,128,128,128]]
        },
        "ActiFun": {
            "values": [ "ReLU","GELU","SELU"]
        },
        "BatchSize": {
            "values": [8,32,64,128]
        },
        "BatchNorm":{
          "values": [True,False]
        },
        "DataAugum":{
          "values": [True,False]
        }
    }
}


sweep_id = wandb.sweep(sweep_config, entity=wandb_entity, project=wandb_project)

wandb: Currently logged in as: ch22m009. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Bhavik More\.netrc
500 response executing GraphQL.
{"errors":[{"message":"Post \"http://anaconda2.default.svc.cluster.local/validate\": read tcp 10.52.64.9:49466-\u003e10.55.247.53:80: read: connection reset by peer","path":["upsertSweep"]}],"data":{"upsertSweep":null}}
wandb: ERROR Error while calling W&B API: Post "http://anaconda2.default.svc.cluster.local/validate": read tcp 10.52.64.9:49466->10.55.247.53:80: read: connection reset by peer (<Response [500]>)


Create sweep with ID: w0blnwlf
Sweep URL: https://wandb.ai/ch22m009/DLASSIGN_2/sweeps/w0blnwlf


**Loading & Splitting Dataset**

In [ ]:
class DataModule(L.LightningDataModule):
    def __init__(self, BatchSize=64, DataAugum= True):
        super().__init__()
        self.BatchSize = BatchSize
        self.DataAugum = DataAugum





        self.transform = transforms.Compose([
              transforms.Resize(size=256),
              transforms.CenterCrop(size=224),
              transforms.ToTensor(),
              transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
          ])

        self.augmentation = transforms.Compose([
              transforms.Resize(256),
              transforms.CenterCrop(224),
              transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
              transforms.RandomAffine(degrees=30, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=10),
              transforms.RandomPerspective(distortion_scale=0.5, p=0.5),
              transforms.RandomErasing(p=0.2, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0),
              transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
              transforms.RandomGrayscale(p=0.1),
              transforms.ToTensor(),
              transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
          ])




    def prepare_data(self):
        self.train_dataset = ImageFolder(root='./nature_12K/inaturalist_12K/train')
        self.test_dataset = ImageFolder(root='./nature_12K/inaturalist_12K/val')



    def setup(self, stage=None):
        dataset = ImageFolder(root='./nature_12K/inaturalist_12K/train')
        test_dataset = ImageFolder(root='./nature_12K/inaturalist_12K/val', transform=self.transform)
        train_size = int(0.8 * len(dataset))
        val_size = len(dataset) - train_size
        train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
        train_dataset.dataset.transform = self.augmentation
        val_dataset.dataset.transform = self.transform
        self.train_dataset, self.val_dataset, self.test_dataset = train_dataset, val_dataset, test_dataset

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.BatchSize, shuffle=True, num_workers=1)


    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.BatchSize, num_workers=1)


    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.BatchSize, num_workers=1)



In [ ]:

class Model1(pl.LightningModule):
    def __init__(self,
                 ActiFun='ReLU',
                 Filters=[32,32,32,32,32],
                 NumDenseNeurons=1024,
                 LearningRate=0.01,
                 BatchSize=64,
                 DropOut=0.5,
                 BatchNorm=True,
                 loss_function='cross_entropy',
                 optimizer='Adam'):
        super().__init__()
        self.save_hyperparameters()

        self.BatchSize = BatchSize

        # Define CNN layers
        cnn_layers = []
        for i in range(5):
            in_channels = 3 if i == 0 else Filters[i - 1]
            cnn_layers.extend([nn.Conv2d(in_channels, Filters[i], kernel_size=3),
                               getattr(nn, ActiFun)()])
            if BatchNorm:
                cnn_layers.append(nn.BatchNorm2d(Filters[i], affine=True))
            cnn_layers.append(nn.MaxPool2d(kernel_size=3, stride=2))

        self.cnn_net = nn.Sequential(*cnn_layers)

        # Calculate output size of CNN layers
        n_sizes = self._get_conv_output((3, 224, 224))

        # Define Dense layers
        dnn_layers = [
            nn.Flatten(),
            nn.Linear(n_sizes, NumDenseNeurons)]
        if DropOut:
            dnn_layers.append(nn.Dropout(DropOut))
        dnn_layers.extend([
            getattr(nn, ActiFun)(),
            nn.Linear(NumDenseNeurons, 10)])
        self.dnn_net = nn.Sequential(*dnn_layers)

        self.accuracy = Accuracy(task="multiclass", num_classes=10)
        self.loss_function = getattr(F, loss_function)
        self.learning_rate = LearningRate
        self.optimizer = optimizer

    def _get_conv_output(self, shape):
        input = torch.autograd.Variable(torch.rand(self.BatchSize, *shape))
        output_feat = self._forward_features(input)
        n_size = output_feat.data.view(self.BatchSize, -1).size(1)
        return n_size

    def _forward_features(self, x):
        return self.cnn_net(x)

    def forward(self, x):
        x = self.cnn_net(x)
        x = self.dnn_net(x)
        return x

    def training_step(self, batch, batch_idx):
        loss, acc = self._common_step(batch)
        self.log('Train_loss', loss, on_step=False, on_epoch=True, logger=True)
        self.log('train_acc', acc, on_step=False, on_epoch=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, acc = self._common_step(batch)
        self.log('Validation_loss', loss, prog_bar=True, on_step=False, on_epoch=True, logger=True)
        self.log('Validation_acc', acc, prog_bar=True, on_step=False, on_epoch=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        loss, acc = self._common_step(batch)
        self.log('Test_loss', loss, prog_bar=True, on_step=False, on_epoch=True, logger=True)
        self.log('Test_acc', acc, prog_bar=True, on_step=False, on_epoch=True, logger=True)
        return loss

    def _common_step(self, batch):
        inputs, target = batch
        output = self.forward(inputs)
        loss = self.loss_function(output, target)
        acc = self.accuracy(output, target)
        return loss, acc

    def configure_optimizers(self):
        return getattr(torch.optim, self.optimizer)(self.parameters(), lr=self.learning_rate)


In [ ]:
def train():

    config_defaults = {
        "Filters": [32,32,32,32,32],
        "BatchNorm": True,
        "DataAugum": True,
        "DropOut":0.5,
        "LearningRate": 1e-3,
        "ActiFun": 'ReLU',
        "BatchSize": 32,
        "Epochs": 10,
        "NumDenseNeurons": 512
    }

    wandb.init(config=config_defaults, magic=True)
    config = wandb.config

    name = "_".join(["Eph",str(config.Epochs),
                 "lr", str(config.LearningRate),
                 "Acti", config.ActiFun,
                 "#filt", str(config.Filters),
                 "DAug", str(config.DataAugum),
                 "DO", str(config.DropOut),
                 "BNrm", str(config.BatchNorm),
                 "#DnsN",str(config.NumDenseNeurons),
                 "BSz",str(config.BatchSize),

                 ])

    data1 = DataModule(config.BatchSize,config.DataAugum)
    data1.prepare_data()
    data1.setup()
    model = Model1(config.ActiFun,config.Filters,config.NumDenseNeurons,config.LearningRate,config.BatchSize,config.DropOut,config.BatchNorm)
    trainer = L.Trainer(max_epochs=config.Epochs)
    trainer.fit(model, data1)


In [ ]:
wandb.agent(sweep_id, train, count=1, entity=wandb_entity, project=wandb_project)
wandb.finish()

wandb: Agent Starting Run: 5gobkgco with config:
wandb: 	ActiFun: GELU
wandb: 	BatchNorm: False
wandb: 	BatchSize: 32
wandb: 	DataAugum: False
wandb: 	DropOut: 0
wandb: 	Epochs: 10
wandb: 	Filters: [64, 64, 128, 128, 256]
wandb: 	LearningRate: 1e-05
wandb: 	NumDenseNeurons: 512
wandb: WARNING wandb.init() arguments ignored because wandb magic has already been initialized


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3050 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type               | Params
------------------------------------------------
0 | cnn_net  | Sequential         | 555 K 
1 | dnn_net  | Sequential         | 2.1 M 
2 | accuracy | MulticlassAccuracy | 0     
------------------------------------------------
2.7 M     Trainable params
0         Non-trainable params
2.7 M     Total params
10.632    Total estimated model params size (MB)


Sanity Checking: |                                                                               | 0/? [00:00<?, ?it/s]

C:\Users\Bhavik More\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


C:\Users\Bhavik More\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Epoch 0:  10%|█████▉                                                        | 24/250 [00:07<01:08,  3.32it/s, v_num=12]